<a href="https://colab.research.google.com/github/leekate/plz/blob/main/petals_to_the_metal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math, re, os
import numpy as np
import tensorflow as tf

print("Tensorflow version " + tf.__version__)

Tensorflow version 2.4.1


In [2]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.94.88.114:8470
INFO:tensorflow:Initializing the TPU system: grpc://10.94.88.114:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.94.88.114:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


# Loading the Competition Data

In [ ]:
# from kaggle_datasets import KaggleDatasets
# GCS_DS_PATH = KaggleDatasets().get_gcs_path('tpu-getting-started')
# print(GCS_DS_PATH)

In [ ]:
#!gsutil ls $GCS_DS_PATH
# you can list the bucket with "!gsutil ls $GCS_DS_PATH"

# from kaggle_datasets import KaggleDatasets
# GCS_DS_PATH = KaggleDatasets().get_gcs_path() 

In [10]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle

In [11]:
!kaggle competitions download -c tpu-getting-started

 60% 27.0M/44.9M [00:00<00:00, 34.4MB/s]
100% 44.9M/44.9M [00:00<00:00, 50.9MB/s]
 90% 41.0M/45.4M [00:00<00:00, 38.9MB/s]
100% 45.4M/45.4M [00:00<00:00, 58.2MB/s]
 95% 42.0M/44.4M [00:00<00:00, 42.1MB/s]
100% 44.4M/44.4M [00:01<00:00, 45.3MB/s]
 89% 41.0M/46.1M [00:00<00:00, 44.6MB/s]
100% 46.1M/46.1M [00:00<00:00, 67.4MB/s]
 90% 41.0M/45.7M [00:01<00:00, 32.7MB/s]
100% 45.7M/45.7M [00:01<00:00, 39.3MB/s]
 82% 37.0M/45.3M [00:00<00:00, 72.0MB/s]
100% 45.3M/45.3M [00:00<00:00, 92.1MB/s]
 71% 32.0M/45.0M [00:00<00:00, 38.7MB/s]
100% 45.0M/45.0M [00:00<00:00, 78.1MB/s]
 73% 33.0M/45.1M [00:00<00:00, 44.5MB/s]
100% 45.1M/45.1M [00:00<00:00, 57.9MB/s]
 72% 33.0M/45.6M [00:00<00:00, 53.0MB/s]
100% 45.6M/45.6M [00:00<00:00, 71.7MB/s]
 54% 25.0M/46.0M [00:00<00:00, 42.9MB/s]
100% 46.0M/46.0M [00:00<00:00, 67.1MB/s]
 72% 33.0M/45.7M [00:01<00:00, 31.5MB/s]
100% 45.7M/45.7M [00:01<00:00, 37.4MB/s]
 84% 37.0M/44.2M [00:00<00:00, 62.4MB/s]
100% 44.2M/44.2M [00:00<00:00, 89.9MB/s]
 90% 41.0M/45.7M

In [12]:
!unzip \*.zip

Archive:  04-512x512-462.tfrec.zip
  inflating: 04-512x512-462.tfrec    

Archive:  06-224x224-462.tfrec.zip
  inflating: 06-224x224-462.tfrec    

Archive:  15-224x224-232.tfrec.zip
  inflating: 15-224x224-232.tfrec    

Archive:  03-512x512-462.tfrec.zip
  inflating: 03-512x512-462.tfrec    

Archive:  02-512x512-232.tfrec.zip
  inflating: 02-512x512-232.tfrec    

Archive:  07-224x224-462.tfrec.zip
  inflating: 07-224x224-462.tfrec    

Archive:  12-224x224-798.tfrec.zip
  inflating: 12-224x224-798.tfrec    

Archive:  10-512x512-462.tfrec.zip
  inflating: 10-512x512-462.tfrec    

Archive:  06-512x512-232.tfrec.zip
  inflating: 06-512x512-232.tfrec    

Archive:  10-512x512-232.tfrec.zip
  inflating: 10-512x512-232.tfrec    

Archive:  02-512x512-798.tfrec.zip
  inflating: 02-512x512-798.tfrec    

Archive:  00-512x512-232.tfrec.zip
  inflating: 00-512x512-232.tfrec    

Archive:  00-512x512-798.tfrec.zip
  inflating: 00-512x512-798.tfrec    

Archive:  00-224x224-462.tfrec.zip
  i

In [13]:
IMAGE_SIZE = [192, 192] # TPU 쓰기
EPOCHS = 5
BATCH_SIZE = 16 * strategy.num_replicas_in_sync

NUM_TRAINING_IMAGES = 12753
NUM_TEST_IMAGES = 7382
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE

In [14]:
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.reshape(image, [*IMAGE_SIZE, 3]) # explicit size needed for TPU
    return image

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "class": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['class'], tf.int32)
    return image, label # returns a dataset of (image, label) pairs

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "id": tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
        # class is missing, this competitions's challenge is to predict flower classes for the test dataset
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['id']
    return image, idnum # returns a dataset of image(s)

def load_dataset(filenames, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.

    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed

    dataset = tf.data.TFRecordDataset(filenames) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord)
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset





```
# def get_training_dataset():
#     dataset = load_dataset(tf.io.gfile.glob(GCS_DS_PATH + '/tfrecords-jpeg-192x192/train/*.tfrec'), labeled=True)
#     dataset = dataset.repeat() # the training dataset must repeat for several epochs
#     dataset = dataset.shuffle(2048)
#     dataset = dataset.batch(BATCH_SIZE)
#     return dataset

# def get_validation_dataset():
#     dataset = load_dataset(tf.io.gfile.glob(GCS_DS_PATH + '/tfrecords-jpeg-192x192/val/*.tfrec'), labeled=True, ordered=False)
#     dataset = dataset.batch(BATCH_SIZE)
#     dataset = dataset.cache()
#     return dataset

# def get_test_dataset(ordered=False):
#     dataset = load_dataset(tf.io.gfile.glob(GCS_DS_PATH + '/tfrecords-jpeg-192x192/test/*.tfrec'), labeled=False, ordered=ordered)
#     dataset = dataset.batch(BATCH_SIZE)
#     return dataset

# training_dataset = get_training_dataset()
# validation_dataset = get_validation_dataset()
```



In [15]:
def get_training_dataset():
    dataset = load_dataset(tf.io.gfile.glob('./tfrecords-jpeg-192x192/train/*.tfrec'), labeled=True)
    dataset = dataset.repeat() # the training dataset must repeat for several epochs
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

def get_validation_dataset():
    dataset = load_dataset(tf.io.gfile.glob('./tfrecords-jpeg-192x192/val/*.tfrec'), labeled=True, ordered=False)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    return dataset

def get_test_dataset(ordered=False):
    dataset = load_dataset(tf.io.gfile.glob('./tfrecords-jpeg-192x192/test/*.tfrec'), labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

training_dataset = get_training_dataset()
validation_dataset = get_validation_dataset()

test_dataset = get_test_dataset()

In [16]:
with strategy.scope():    
    pretrained_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False ,input_shape=[*IMAGE_SIZE, 3])
    pretrained_model.trainable = False # tramsfer learning
    
    model = tf.keras.Sequential([
        pretrained_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(104, activation='softmax')
    ])
        
model.compile(
    optimizer='adam',
    loss = 'categorical_crossentropy',
    metrics=['categorical_accuracy']
)

In [18]:
training_dataset

<BatchDataset shapes: ((None, 192, 192, 3), (None,)), types: (tf.float32, tf.int32)>

In [19]:
validation_dataset

<CacheDataset shapes: ((None, 192, 192, 3), (None,)), types: (tf.float32, tf.int32)>

In [17]:
historical = model.fit(training_dataset, 
          steps_per_epoch=STEPS_PER_EPOCH, 
          epochs=EPOCHS, 
          validation_data=validation_dataset)

Epoch 1/5


ValueError: ignored

# Test

In [ ]:
test_ds = get_test_dataset(ordered=True) 

print('Computing predictions...')
test_images_ds = test_ds.map(lambda image, idnum: image)
probabilities = model.predict(test_images_ds)
predictions = np.argmax(probabilities, axis=-1)
print(predictions)

print('Generating submission.csv file...')
test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U') # all in one batch
np.savetxt('submission.csv', np.rec.fromarrays([test_ids, predictions]), fmt=['%s', '%d'], delimiter=',', header='id,label', comments='')